In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# las librerias que mas se necesitan al menos en este proyecto serian pandas (aunque se unicamente para limpiar los datos nulos y sklearn que es la libreria que nos va a guiar en el proceso
# de machine learning)
#matplotlib.pyplot es para graficar los datos por lo que si se pudiera hacer en C# seria super util pero no es 100% indispensable
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

Lectura de datos directo del archivo llamado 'Expanded_data_with_more_features.csv' y luego entra a datos donde con pandas se eliminan las filas donde en las columnas del genero, testprep y practice sport son valores nulos, esto debido a que luego se va a preparar estas columnas para que sean variables discretas y se pueda ajustar al modelo de regrecion lineal. 

In [30]:
daticos = pd.read_csv('Expanded_data_with_more_features.csv')

In [31]:
datos = daticos.dropna(subset=['Gender','TestPrep','PracticeSport', 'WklyStudyHours','MathScore','ReadingScore','WritingScore'])

In [32]:
datos.head()

,Unnamed: 0,Gender,EthnicGroup,ParentEduc,LunchType,TestPrep,ParentMaritalStatus,PracticeSport,IsFirstChild,NrSiblings,TransportMeans,WklyStudyHours,MathScore,ReadingScore,WritingScore
0,0,female,NaN,bachelor's degree,standard,none,married,regularly,yes,3.0,school_bus,< 5,71,71,74
2,2,female,group B,master's degree,standard,none,single,sometimes,yes,4.0,school_bus,< 5,87,93,91
3,3,male,group A,associate's degree,free/reduced,none,married,never,no,1.0,NaN,5 - 10,45,56,42
4,4,male,group C,some college,standard,none,married,sometimes,yes,0.0,school_bus,5 - 10,76,78,75
5,5,female,group B,associate's degree,standard,none,married,regularly,yes,1.0,school_bus,5 - 10,73,84,79


In [48]:
datos.WklyStudyHours.value_counts()

5 - 10    14936
< 5        7588
> 10       4805
Name: WklyStudyHours, dtype: int64

In [33]:
daticos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30641 entries, 0 to 30640
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           30641 non-null  int64  
 1   Gender               30641 non-null  object 
 2   EthnicGroup          28801 non-null  object 
 3   ParentEduc           28796 non-null  object 
 4   LunchType            30641 non-null  object 
 5   TestPrep             28811 non-null  object 
 6   ParentMaritalStatus  29451 non-null  object 
 7   PracticeSport        30010 non-null  object 
 8   IsFirstChild         29737 non-null  object 
 9   NrSiblings           29069 non-null  float64
 10  TransportMeans       27507 non-null  object 
 11  WklyStudyHours       29686 non-null  object 
 12  MathScore            30641 non-null  int64  
 13  ReadingScore         30641 non-null  int64  
 14  WritingScore         30641 non-null  int64  
dtypes: float64(1), int64(4), object(10)


In [34]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27329 entries, 0 to 30640
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           27329 non-null  int64  
 1   Gender               27329 non-null  object 
 2   EthnicGroup          25718 non-null  object 
 3   ParentEduc           25695 non-null  object 
 4   LunchType            27329 non-null  object 
 5   TestPrep             27329 non-null  object 
 6   ParentMaritalStatus  26261 non-null  object 
 7   PracticeSport        27329 non-null  object 
 8   IsFirstChild         26526 non-null  object 
 9   NrSiblings           25927 non-null  float64
 10  TransportMeans       24549 non-null  object 
 11  WklyStudyHours       27329 non-null  object 
 12  MathScore            27329 non-null  int64  
 13  ReadingScore         27329 non-null  int64  
 14  WritingScore         27329 non-null  int64  
dtypes: float64(1), int64(4), object(10)


En este punto se usa 'One Hot Encoding' para convertir las variables categoricas en variables discretas, esta funcion nace de scklearn, por lo que seria necesario encontrar una libreria que me permitiera hacer esto o en su defecto poder hacerlo manualmente sin que sea tan engorroso.

In [35]:
datos.Gender.value_counts()

female    13774
male      13555
Name: Gender, dtype: int64

In [36]:
codificador = OneHotEncoder()

In [37]:
genero_codificado = codificador.fit_transform(datos[['Gender']])
genero_codificado_df = pd.DataFrame(genero_codificado.toarray(), columns=codificador.get_feature_names_out(['Gender']))
test_codificado = codificador.fit_transform(datos[['TestPrep']])
test_codificado_df = pd.DataFrame(test_codificado.toarray(), columns=codificador.get_feature_names_out(['TestPrep']))
deporte_codificado = codificador.fit_transform(datos[['PracticeSport']])
deporte_codificado_df = pd.DataFrame(deporte_codificado.toarray(), columns=codificador.get_feature_names_out(['PracticeSport']))

Como se ve acontinuacion, el genero por ejemplo se combierte en 2 columnas donde si es femenino pasa a valer 1, de lo contrario 0, si es masculino vale 1 de lo contrario 0. 

In [38]:
genero_codificado_df.head()

,Gender_female,Gender_male
0,1.0,0.0
1,1.0,0.0
2,0.0,1.0
3,0.0,1.0
4,1.0,0.0


In [39]:
test_codificado_df.head()

,TestPrep_completed,TestPrep_none
0,0.0,1.0
1,0.0,1.0
2,0.0,1.0
3,0.0,1.0
4,0.0,1.0


In [40]:
deporte_codificado_df.head()

,PracticeSport_never,PracticeSport_regularly,PracticeSport_sometimes
0,0.0,1.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,0.0,0.0,1.0
4,0.0,1.0,0.0


ahora voy a preparar los datos, donde ya teniendo los dataframes limpios para hacer el modelo se necesitan tener en un solo dataframe y no en 3 como esta hasta este momento, asi que tocaria hacer un concat y listo

In [41]:
variables_categoricas = pd.concat([genero_codificado_df, test_codificado_df], axis=1)

In [42]:
variables_categoricas.head()

,Gender_female,Gender_male,TestPrep_completed,TestPrep_none
0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0
2,0.0,1.0,0.0,1.0
3,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,1.0


In [43]:
variable_objetivo = datos[['MathScore',]]

In [44]:
variable_objetivo.head()

,MathScore
0,71
2,87
3,45
4,76
5,73


In [45]:
datox = pd.concat([variables_categoricas, variable_objetivo], axis=1)

In [46]:
datox

,Gender_female,Gender_male,TestPrep_completed,TestPrep_none,MathScore
0,1.0,0.0,0.0,1.0,71.0
1,1.0,0.0,0.0,1.0,NaN
2,0.0,1.0,0.0,1.0,87.0
3,0.0,1.0,0.0,1.0,45.0
4,1.0,0.0,0.0,1.0,76.0
...,...,...,...,...,...
30636,NaN,NaN,NaN,NaN,59.0
30637,NaN,NaN,NaN,NaN,58.0
30638,NaN,NaN,NaN,NaN,61.0
30639,NaN,NaN,NaN,NaN,82.0


In [47]:
np.corrcoef(datox, rowvar=False)

array([[nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan],
       [nan, nan, nan, nan, nan]])